## Environment Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
%cd rapidsai-csp-utils
!git checkout 22.12-patch
%cd ..
!python rapidsai-csp-utils/colab/env-check.py
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.11 KiB | 2.10 MiB/s, done.
Resolving deltas: 100% (191/191), done.
/content/rapidsai-csp-utils
Branch '22.12-patch' set up to track remote branch '22.12-patch' from 'origin'.
Switched to a new branch '22.12-patch'
/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.2 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/env-check.py", line 26, in <module>
    gpu_name = pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0)).decode('UTF-8')
AttributeError: 'str' object has no attribute 'decode'. Did you mean: 'encode'?
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 26, in <module>
    gpu_name =

In [3]:
%%bash
# Install the Merlin Framework
pip install -Uq git+https://github.com/NVIDIA-Merlin/models.git@release-23.04
pip install -Uq git+https://github.com/NVIDIA-Merlin/nvtabular.git@release-23.04
pip install -Uq git+https://github.com/NVIDIA-Merlin/core.git@release-23.04
# pip install -Uq git+https://github.com/NVIDIA-Merlin/system.git@release-23.04
# pip install -Uq git+https://github.com/NVIDIA-Merlin/dataloader.git@release-23.04
pip install -Uq git+https://github.com/NVIDIA-Merlin/Transformers4Rec.git@release-23.04
# pip install -Uq xgboost lightfm implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.5/985.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
merlin-dataloader 23.6.0 requires merlin-core<23.07,>=23.06, but you have merlin-core 23.4.0+4.gc5c9bc2 which is incompatible.


In [4]:
%%bash
pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.0 MB/s eta 0:00:00


## Load Data
Reference
- https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb

In [5]:
import os
import shutil
import numpy as np
from nvtabular import ops
from merlin.schema.tags import Tags
import merlin.io
import merlin.models.tf as mm

import nvtabular as nvt

from os import path

# Get dataframe library - cudf or pandas
from merlin.core.dispatch import get_lib
pd = get_lib()

/usr/local/lib/python3.10/dist-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


In [6]:
DATA_DIR = os.environ.get(
    "DATA_DIR", os.path.expanduser("/content/drive/MyDrive/ml-twotower-model/data/")
)

In [7]:
import glob

USE_AUGMENTED_DATASET= False
DATA_VERSION = 'v4'

_TRAIN,_VALID,_TEST = ('train_aug', 'valid_aug', 'test_aug') if USE_AUGMENTED_DATASET else ('train', 'valid', 'test')

TRAIN_PATHS = sorted(glob.glob(os.path.join(DATA_DIR, _TRAIN + f'_{DATA_VERSION}', "*.parquet")))
VALID_PATHS = sorted(glob.glob(os.path.join(DATA_DIR, _VALID + f'_{DATA_VERSION}', "*.parquet")))
TEST_PATHS = sorted(glob.glob(os.path.join(DATA_DIR, _TEST + f'_{DATA_VERSION}', "*.parquet")))
TRAIN_PATHS, VALID_PATHS, TEST_PATHS

(['/content/drive/MyDrive/ml-twotower-model/data/train_v4/part_0.parquet'],
 ['/content/drive/MyDrive/ml-twotower-model/data/valid_v4/part_0.parquet'],
 ['/content/drive/MyDrive/ml-twotower-model/data/test_v4/part_0.parquet'])

In [8]:
train = merlin.io.Dataset(
    TRAIN_PATHS, engine="parquet"
)
valid = merlin.io.Dataset(
    VALID_PATHS, engine="parquet"
)

/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [9]:
schema = train.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER, Tags.TARGET])
schema

,name,tags,dtype,is_list,is_ragged,properties.start_index,properties.max_size,properties.freq_threshold,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.num_buckets,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,movie_id,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,0.0,0.0,0.0,1665.0,102.0,NaN,.//categories/unique.movie_id.parquet,0.0,1664.0,movie_id,NaN,NaN
1,user_id,"(Tags.ID, Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,0.0,0.0,0.0,944.0,74.0,NaN,.//categories/unique.user_id.parquet,0.0,943.0,user_id,NaN,NaN
2,gender,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,0.0,0.0,0.0,3.0,16.0,NaN,.//categories/unique.gender.parquet,0.0,2.0,gender,NaN,NaN
3,occupation,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,0.0,0.0,0.0,22.0,16.0,NaN,.//categories/unique.occupation.parquet,0.0,21.0,occupation,NaN,NaN
4,genres,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,0.0,0.0,0.0,20.0,16.0,NaN,.//categories/unique.genres.parquet,0.0,19.0,genres,0.0,NaN
5,rating,"(Tags.TARGET, Tags.BINARY_CLASSIFICATION)","DType(name='float32', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
train.to_ddf().compute()

,movie_id,user_id,gender,occupation,genres,rating,movie_id_raw
0,121,13,1,1,[1],1.0,134
1,196,33,1,18,"[1, 4, 8]",1.0,31
2,451,901,1,1,"[2, 5]",1.0,354
3,48,88,2,8,"[2, 5]",0.0,202
4,330,30,1,4,[1],0.0,708
...,...,...,...,...,...,...,...
70197,60,232,1,1,"[1, 13]",0.0,289
70198,89,80,1,1,"[4, 3, 9]",0.0,273
70199,180,253,1,11,[1],1.0,285
70200,507,111,1,15,[1],1.0,468


In [11]:
# select only trainable features and exclude the target
train.schema = schema
valid.schema = schema

In [12]:
assert len(schema.select_by_tag(Tags.TARGET).column_names) == 0

AssertionError: ignored

## Build Model
Reference
- https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb

### Model definition

In [13]:
def build_towers(schema=schema, tower_dim = 64):
    # create user schema using USER tag
    user_schema = schema.select_by_tag(Tags.USER)
    # create user (query) tower input block
    user_inputs = mm.InputBlockV2(user_schema)
    # create user (query) encoder block
    query = mm.Encoder(user_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

    # create item schema using ITEM tag
    item_schema = schema.select_by_tag(Tags.ITEM)
    # create item (candidate) tower input block
    item_inputs = mm.InputBlockV2(item_schema)
    # create item (candidate) encoder block
    candidate = mm.Encoder(item_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

    return mm.TwoTowerModelV2(query, candidate)


### model_v1

In [16]:
model = build_towers()

In [17]:
import tensorflow as tf
opt = tf.keras.optimizers.Adagrad(learning_rate=1e-2)
model.compile(optimizer=opt, run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC()])
history = model.fit(train, validation_data=valid, batch_size=128, epochs=10)

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed i

Epoch 1/10
549/549 [==============================] - 34s 41ms/step - loss: 4.8493 - auc: 0.4999 - recall_at_10: 0.0801 - ndcg_at_10: 0.0371 - regularization_loss: 0.0000e+00 - loss_batch: 4.8471 - val_loss: 4.8488 - val_auc: 0.5002 - val_recall_at_10: 0.0790 - val_ndcg_at_10: 0.0361 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.5984
Epoch 2/10
549/549 [==============================] - 13s 22ms/step - loss: 4.8494 - auc: 0.5000 - recall_at_10: 0.0813 - ndcg_at_10: 0.0374 - regularization_loss: 0.0000e+00 - loss_batch: 4.8471 - val_loss: 4.8488 - val_auc: 0.5002 - val_recall_at_10: 0.0788 - val_ndcg_at_10: 0.0361 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.5984
Epoch 3/10
549/549 [==============================] - 14s 23ms/step - loss: 4.8493 - auc: 0.5000 - recall_at_10: 0.0816 - ndcg_at_10: 0.0376 - regularization_loss: 0.0000e+00 - loss_batch: 4.8471 - val_loss: 4.8487 - val_auc: 0.5001 - val_recall_at_10: 0.0803 - val_ndcg_at_10: 0.0366 - val_regularizatio

### model_v2

In [19]:
model_v2 = build_towers()

In [20]:
import tensorflow as tf
opt = tf.keras.optimizers.Adam(learning_rate=1e-2)
model_v2.compile(optimizer=opt, run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC(), mm.MRRAt(10), mm.PrecisionAt(10)])
history = model_v2.fit(train, validation_data=valid, batch_size=128, epochs=50)

Epoch 1/50
549/549 [==============================] - 31s 42ms/step - loss: 4.7178 - auc_3: 0.5832 - recall_at_10: 0.1140 - ndcg_at_10: 0.0533 - mrr_at_10: 0.0353 - precision_at_10: 0.0114 - regularization_loss: 0.0000e+00 - loss_batch: 4.7155 - val_loss: 4.7411 - val_auc_3: 0.6028 - val_recall_at_10: 0.1718 - val_ndcg_at_10: 0.0811 - val_mrr_at_10: 0.0542 - val_precision_at_10: 0.0172 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.8512
Epoch 2/50
549/549 [==============================] - 15s 26ms/step - loss: 4.4944 - auc_3: 0.6298 - recall_at_10: 0.2186 - ndcg_at_10: 0.1048 - mrr_at_10: 0.0709 - precision_at_10: 0.0219 - regularization_loss: 0.0000e+00 - loss_batch: 4.4922 - val_loss: 4.6490 - val_auc_3: 0.5609 - val_recall_at_10: 0.1874 - val_ndcg_at_10: 0.0910 - val_mrr_at_10: 0.0621 - val_precision_at_10: 0.0187 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.6053
Epoch 3/50
549/549 [==============================] - 16s 28ms/step - loss: 4.4325 - auc_3: 0.63

In [29]:
model_v2.fit(train, validation_data=valid, batch_size=128, epochs=100, initial_epoch=len(model_v2.history.epoch))

Epoch 52/100
549/549 [==============================] - 14s 24ms/step - loss: 4.2580 - auc_3: 0.6103 - recall_at_10: 0.3128 - ndcg_at_10: 0.1681 - mrr_at_10: 0.1244 - precision_at_10: 0.0313 - regularization_loss: 0.0000e+00 - loss_batch: 4.2557 - val_loss: 4.7088 - val_auc_3: 0.5440 - val_recall_at_10: 0.2060 - val_ndcg_at_10: 0.1006 - val_mrr_at_10: 0.0691 - val_precision_at_10: 0.0206 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 5.1694
Epoch 53/100
549/549 [==============================] - 15s 26ms/step - loss: 4.2522 - auc_3: 0.6070 - recall_at_10: 0.3133 - ndcg_at_10: 0.1681 - mrr_at_10: 0.1243 - precision_at_10: 0.0313 - regularization_loss: 0.0000e+00 - loss_batch: 4.2500 - val_loss: 4.6901 - val_auc_3: 0.5508 - val_recall_at_10: 0.2080 - val_ndcg_at_10: 0.1003 - val_mrr_at_10: 0.0682 - val_precision_at_10: 0.0208 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 5.1030
Epoch 54/100
549/549 [==============================] - 14s 24ms/step - loss: 4.2546 - auc_3

### model_v3

In [40]:
model_v3 = build_towers(tower_dim=256)

In [41]:
model_v3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
                 run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC(), mm.MRRAt(10), mm.PrecisionAt(10)])
model_v3.fit(train, validation_data=valid, batch_size=128, epochs=50)

Epoch 1/50
549/549 [==============================] - 29s 43ms/step - loss: 4.7934 - auc_9: 0.5473 - recall_at_10: 0.0959 - ndcg_at_10: 0.0448 - mrr_at_10: 0.0297 - precision_at_10: 0.0096 - regularization_loss: 0.0000e+00 - loss_batch: 4.7909 - val_loss: 4.7574 - val_auc_9: 0.5705 - val_recall_at_10: 0.1465 - val_ndcg_at_10: 0.0679 - val_mrr_at_10: 0.0446 - val_precision_at_10: 0.0147 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.7600
Epoch 2/50
549/549 [==============================] - 12s 21ms/step - loss: 4.5831 - auc_9: 0.6291 - recall_at_10: 0.1819 - ndcg_at_10: 0.0869 - mrr_at_10: 0.0586 - precision_at_10: 0.0182 - regularization_loss: 0.0000e+00 - loss_batch: 4.5807 - val_loss: 4.6974 - val_auc_9: 0.5981 - val_recall_at_10: 0.1775 - val_ndcg_at_10: 0.0832 - val_mrr_at_10: 0.0552 - val_precision_at_10: 0.0178 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.6087
Epoch 3/50
549/549 [==============================] - 15s 26ms/step - loss: 4.4773 - auc_9: 0.64

### model_v4

In [43]:
model_v4 = build_towers(tower_dim=64)

In [44]:
model_v4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
                 run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC(), mm.MRRAt(10), mm.PrecisionAt(10)])
model_v4.fit(train, validation_data=valid, batch_size=64, epochs=50)

Epoch 1/50
1097/1097 [==============================] - 41s 30ms/step - loss: 4.0937 - auc_10: 0.5621 - recall_at_10: 0.1918 - ndcg_at_10: 0.0882 - mrr_at_10: 0.0575 - precision_at_10: 0.0192 - regularization_loss: 0.0000e+00 - loss_batch: 4.0935 - val_loss: 4.0448 - val_auc_10: 0.6044 - val_recall_at_10: 0.2650 - val_ndcg_at_10: 0.1258 - val_mrr_at_10: 0.0842 - val_precision_at_10: 0.0265 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 3.7124
Epoch 2/50
1097/1097 [==============================] - 22s 20ms/step - loss: 3.9356 - auc_10: 0.6167 - recall_at_10: 0.2999 - ndcg_at_10: 0.1441 - mrr_at_10: 0.0975 - precision_at_10: 0.0300 - regularization_loss: 0.0000e+00 - loss_batch: 3.9356 - val_loss: 3.9807 - val_auc_10: 0.5899 - val_recall_at_10: 0.3061 - val_ndcg_at_10: 0.1483 - val_mrr_at_10: 0.1012 - val_precision_at_10: 0.0306 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 3.6598
Epoch 3/50
1097/1097 [==============================] - 25s 22ms/step - loss: 3.8501 - a

### model_v5

In [14]:
model_v5 = build_towers(tower_dim=64)

In [26]:
TOK_K = 10
model_v5.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
                 run_eagerly=False, metrics=[mm.TopKMetricsAggregator.default_metrics(top_ks=TOP_K)],
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
)
model_v5.fit(train, validation_data=valid, batch_size=32, epochs=20)

Epoch 1/20
2194/2194 [==============================] - 61s 25ms/step - loss: 3.4274 - recall_at_10: 0.3553 - mrr_at_10: 0.1066 - ndcg_at_10: 0.1635 - map_at_10: 0.1066 - precision_at_10: 0.0355 - regularization_loss: 0.0000e+00 - loss_batch: 3.4272 - val_loss: 3.3944 - val_recall_at_10: 0.4311 - val_mrr_at_10: 0.1346 - val_ndcg_at_10: 0.2026 - val_map_at_10: 0.1346 - val_precision_at_10: 0.0431 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.6066
Epoch 2/20
2194/2194 [==============================] - 51s 23ms/step - loss: 3.3122 - recall_at_10: 0.4674 - mrr_at_10: 0.1545 - ndcg_at_10: 0.2266 - map_at_10: 0.1545 - precision_at_10: 0.0467 - regularization_loss: 0.0000e+00 - loss_batch: 3.3121 - val_loss: 3.3180 - val_recall_at_10: 0.4833 - val_mrr_at_10: 0.1615 - val_ndcg_at_10: 0.2357 - val_map_at_10: 0.1615 - val_precision_at_10: 0.0483 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.3734
Epoch 3/20
2194/2194 [==============================] - 50s 23ms/step - loss

### model_v6

In [21]:
model_v6 = build_towers(tower_dim=256)

In [22]:
TOK_K = 10
model_v6.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
                 run_eagerly=False, metrics=[mm.TopKMetricsAggregator.default_metrics(top_ks=TOP_K)])
model_v6.fit(train, validation_data=valid, batch_size=32, epochs=20)

Epoch 1/20
2194/2194 [==============================] - 64s 26ms/step - loss: 3.4650 - recall_at_10: 0.3208 - mrr_at_10: 0.0946 - ndcg_at_10: 0.1462 - map_at_10: 0.0946 - precision_at_10: 0.0321 - regularization_loss: 0.0000e+00 - loss_batch: 3.4648 - val_loss: 3.4638 - val_recall_at_10: 0.3136 - val_mrr_at_10: 0.0923 - val_ndcg_at_10: 0.1429 - val_map_at_10: 0.0923 - val_precision_at_10: 0.0314 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.2817
Epoch 2/20
2194/2194 [==============================] - 54s 24ms/step - loss: 3.4638 - recall_at_10: 0.3138 - mrr_at_10: 0.0926 - ndcg_at_10: 0.1431 - map_at_10: 0.0926 - precision_at_10: 0.0314 - regularization_loss: 0.0000e+00 - loss_batch: 3.4637 - val_loss: 3.4637 - val_recall_at_10: 0.3133 - val_mrr_at_10: 0.0922 - val_ndcg_at_10: 0.1427 - val_map_at_10: 0.0922 - val_precision_at_10: 0.0313 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.2815
Epoch 3/20
2194/2194 [==============================] - 47s 21ms/step - loss

In [23]:
model_v6.fit(train, validation_data=valid, batch_size=32, epochs=30, initial_epoch=20)

Epoch 21/30
2194/2194 [==============================] - 50s 22ms/step - loss: 3.4637 - recall_at_10: 0.7700 - mrr_at_10: 0.5500 - ndcg_at_10: 0.6052 - map_at_10: 0.5500 - precision_at_10: 0.0770 - regularization_loss: 0.0000e+00 - loss_batch: 3.4636 - val_loss: 3.4636 - val_recall_at_10: 0.0449 - val_mrr_at_10: 0.0237 - val_ndcg_at_10: 0.0290 - val_map_at_10: 0.0237 - val_precision_at_10: 0.0045 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.2815
Epoch 22/30
2194/2194 [==============================] - 49s 22ms/step - loss: 3.4638 - recall_at_10: 0.7736 - mrr_at_10: 0.5556 - ndcg_at_10: 0.6104 - map_at_10: 0.5556 - precision_at_10: 0.0774 - regularization_loss: 0.0000e+00 - loss_batch: 3.4636 - val_loss: 3.4638 - val_recall_at_10: 1.0000 - val_mrr_at_10: 0.5492 - val_ndcg_at_10: 0.6623 - val_map_at_10: 0.5492 - val_precision_at_10: 0.1000 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.2815
Epoch 23/30
2194/2194 [==============================] - 46s 21ms/step - l

### model_v7

In [52]:
model_v7 = build_towers(tower_dim=256)

In [53]:
model_v7.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-2),
                 run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10), tf.keras.metrics.AUC(), mm.MRRAt(10), mm.PrecisionAt(10)])
model_v7.fit(train, validation_data=valid, batch_size=32, epochs=20)

Epoch 1/20
2194/2194 [==============================] - 57s 23ms/step - loss: 3.7826 - auc_14: 0.5041 - recall_at_10: 0.4627 - ndcg_at_10: 0.3637 - mrr_at_10: 0.3345 - precision_at_10: 0.0463 - regularization_loss: 0.0000e+00 - loss_batch: 3.7823 - val_loss: 3.4636 - val_auc_14: 0.5028 - val_recall_at_10: 0.1396 - val_ndcg_at_10: 0.1372 - val_mrr_at_10: 0.1364 - val_precision_at_10: 0.0140 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.2815
Epoch 2/20
2194/2194 [==============================] - 45s 20ms/step - loss: 3.4637 - auc_14: 0.5026 - recall_at_10: 0.6896 - ndcg_at_10: 0.6859 - mrr_at_10: 0.6846 - precision_at_10: 0.0690 - regularization_loss: 0.0000e+00 - loss_batch: 3.4635 - val_loss: 3.4635 - val_auc_14: 0.5023 - val_recall_at_10: 0.9311 - val_ndcg_at_10: 0.9306 - val_mrr_at_10: 0.9304 - val_precision_at_10: 0.0931 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.2815
Epoch 3/20
2194/2194 [==============================] - 45s 20ms/step - loss: 3.4638 - a

In [ ]:
model_v7.predict()

In [55]:
predict_v7 = model_v7.batch_predict(valid, batch_size=128)

/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


ValueError: ignored

In [76]:
model_v3 = build_towers()

In [75]:
import tensorflow_recommenders as tfrs
topk_metric = tfrs.metrics.FactorizedTopK(
  candidates=tf.data.Dataset.from_tensor_slices(train.to_ddf().compute()['movie_id'].unique())
)
# hack
setattr(topk_metric, 'reset_state', topk_metric.reset_states)

In [77]:
import tensorflow as tf
opt = tf.keras.optimizers.Adam(learning_rate=1e-2)
model_v3.compile(optimizer=opt, run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10),
                                                            tf.keras.metrics.AUC(), mm.MRRAt(10),
                                                            topk_metric
                                                            ])
history = model_v3.fit(train, validation_data=valid, batch_size=32, epochs=10)

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed i

Epoch 1/10
2194/2194 [==============================] - 62s 26ms/step - loss: 3.4275 - auc_8: 0.5490 - factorized_top_k: 0.0303 - recall_at_10: 0.3520 - ndcg_at_10: 0.1636 - mrr_at_10: 0.1077 - regularization_loss: 0.0000e+00 - loss_batch: 3.4274 - val_loss: 3.4139 - val_auc_8: 0.5758 - val_factorized_top_k: 0.0303 - val_recall_at_10: 0.4113 - val_ndcg_at_10: 0.1926 - val_mrr_at_10: 0.1274 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.3728
Epoch 2/10
2194/2194 [==============================] - 51s 23ms/step - loss: 3.3574 - auc_8: 0.5811 - factorized_top_k: 0.0303 - recall_at_10: 0.4486 - ndcg_at_10: 0.2135 - mrr_at_10: 0.1433 - regularization_loss: 0.0000e+00 - loss_batch: 3.3572 - val_loss: 3.3950 - val_auc_8: 0.5729 - val_factorized_top_k: 0.0303 - val_recall_at_10: 0.4260 - val_ndcg_at_10: 0.2004 - val_mrr_at_10: 0.1331 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 2.4693
Epoch 3/10
2194/2194 [==============================] - 50s 23ms/step - loss: 3.3413 - a

## Evaluate the Model top-k results

In [67]:
from mm.metrics.topk.


ImportError: ignored

In [52]:
# Top-K evaluation
from merlin.models.utils.dataset import unique_rows_by_features

candidate_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
candidate_features.head()

/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


,movie_id,genres
0,9,"[14, 2, 10]"
1,231,"[7, 2, 17, 5]"
2,544,[9]
3,484,"[1, 4]"
4,10,"[1, 9, 3]"


In [61]:
candidate_features.to_ddf()['genres'][0].compute().values

array([array([14,  2, 10])], dtype=object)

In [53]:
topk = 20
topk_model = model.to_top_k_encoder(candidate_features, k=topk, batch_size=128)

# we can set `metrics` param in the `compile(), if we want
topk_model.compile(run_eagerly=False)

/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/io/dataset.py:264: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


ValueError: ignored

## TMP

In [ ]:
# model = mm.DLRMModel(
#     train.schema,
#     embedding_dim=64,
#     bottom_block=mm.MLPBlock([128, 64]),
#     top_block=mm.MLPBlock([128, 64, 32]),
#     prediction_tasks=mm.BinaryOutput(
#         train.schema.select_by_tag(Tags.TARGET).column_names[0]
#     ),
# )
# model.compile(optimizer="adam")
# model.fit(train, batch_size=1024)

In [9]:
schema

,name,tags,dtype,is_list,is_ragged,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.cat_path,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,movie_id,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,1665.0,102.0,.//categories/unique.movie_id.parquet,NaN,0.0,0.0,0.0,0.0,1664.0,movie_id,NaN,NaN
1,user_id,"(Tags.ID, Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,944.0,74.0,.//categories/unique.user_id.parquet,NaN,0.0,0.0,0.0,0.0,943.0,user_id,NaN,NaN
2,gender,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,3.0,16.0,.//categories/unique.gender.parquet,NaN,0.0,0.0,0.0,0.0,2.0,gender,NaN,NaN
3,occupation,"(Tags.USER, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",False,False,22.0,16.0,.//categories/unique.occupation.parquet,NaN,0.0,0.0,0.0,0.0,21.0,occupation,NaN,NaN
4,genres,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,20.0,16.0,.//categories/unique.genres.parquet,NaN,0.0,0.0,0.0,0.0,19.0,genres,0.0,NaN
5,rating,"(Tags.TARGET, Tags.BINARY_CLASSIFICATION)","DType(name='float32', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Define Model
import tensorflow as tf
QUERY_DROPOUT = 0.1
ITEM_DROPOUT = 0.1

model_v0 = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock(
        [128,256],
        kernel_regularizer=tf.keras.regularizers.l1(3.9e-06),
        bias_regularizer=tf.keras.regularizers.l1(3.9e-06),
        no_activation_last_layer=True,
        # dropout=QUERY_DROPOUT
     ),
     item_tower=mm.MLPBlock(
         [128,256],
         kernel_regularizer=tf.keras.regularizers.l1(3.9e-06),
         bias_regularizer=tf.keras.regularizers.l1(3.9e-06),
         no_activation_last_layer=True,
        #  dropout=ITEM_DROPOUT
     ),
     samplers=[mm.InBatchSampler()] # default value
)

In [14]:
TOP_K = [10]

model_v0.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.02),
    run_eagerly=False,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[mm.TopKMetricsAggregator.default_metrics(top_ks=TOP_K),
            #  tfr.keras.metrics.MRRMetric(topn=4)
             ],
)
model_v0.fit(
    train,
    validation_data=valid,
    schema=schema,
    batch_size=32,
    epochs=10,
    # shuffle = False
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


2194/2194 [==============================] - ETA: 0s - loss: 4.3563 - recall_at_10: 0.3120 - mrr_at_10: 0.0964 - ndcg_at_10: 0.1456 - map_at_10: 0.0964 - precision_at_10: 0.0312 - regularization_loss: 0.0469 - loss_batch: 4.3562

2194/2194 [==============================] - 64s 26ms/step - loss: 4.3563 - recall_at_10: 0.3120 - mrr_at_10: 0.0964 - ndcg_at_10: 0.1456 - map_at_10: 0.0964 - precision_at_10: 0.0312 - regularization_loss: 0.0469 - loss_batch: 4.3557 - val_loss: 3.5576 - val_recall_at_10: 0.4510 - val_mrr_at_10: 0.2752 - val_ndcg_at_10: 0.3147 - val_map_at_10: 0.2752 - val_precision_at_10: 0.0451 - val_regularization_loss: 0.0635 - val_loss_batch: 2.4614
Epoch 2/10
2194/2194 [==============================] - 58s 26ms/step - loss: 7.0354 - recall_at_10: 0.3588 - mrr_at_10: 0.1765 - ndcg_at_10: 0.2181 - map_at_10: 0.1765 - precision_at_10: 0.0359 - regularization_loss: 0.1229 - loss_batch: 7.0334 - val_loss: 3.6345 - val_recall_at_10: 0.3021 - val_mrr_at_10: 0.0890 - val_ndcg_at_10: 0.1377 - val_map_at_10: 0.0890 - val_precision_at_10: 0.0302 - val_regularization_loss: 0.1405 - val_loss_batch: 2.5384
Epoch 3/10
2194/2194 [==============================] - 51s 23ms/step - loss: 3.7440 - recall_at_10: 0.

In [15]:
model_v0.fit(
    train,
    validation_data=valid,
    schema=schema,
    batch_size=32,
    epochs=20,
    initial_epoch=10
    # shuffle = False
)

Epoch 11/20
2194/2194 [==============================] - 49s 22ms/step - loss: 4.3707 - recall_at_10: 0.2994 - mrr_at_10: 0.0776 - ndcg_at_10: 0.1282 - map_at_10: 0.0776 - precision_at_10: 0.0299 - regularization_loss: 0.1081 - loss_batch: 4.3701 - val_loss: 3.6014 - val_recall_at_10: 0.2994 - val_mrr_at_10: 0.0769 - val_ndcg_at_10: 0.1277 - val_map_at_10: 0.0769 - val_precision_at_10: 0.0299 - val_regularization_loss: 0.1071 - val_loss_batch: 2.5050
Epoch 12/20
2194/2194 [==============================] - 50s 22ms/step - loss: 4.2492 - recall_at_10: 0.2999 - mrr_at_10: 0.0775 - ndcg_at_10: 0.1283 - map_at_10: 0.0775 - precision_at_10: 0.0300 - regularization_loss: 0.1018 - loss_batch: 4.2488 - val_loss: 3.5974 - val_recall_at_10: 0.2982 - val_mrr_at_10: 0.0765 - val_ndcg_at_10: 0.1271 - val_map_at_10: 0.0765 - val_precision_at_10: 0.0298 - val_regularization_loss: 0.1031 - val_loss_batch: 2.5013
Epoch 13/20
2194/2194 [==============================] - 56s 25ms/step - loss: 5.1365 - re

In [16]:
model_v0.fit(
    train,
    validation_data=valid,
    schema=schema,
    batch_size=32,
    epochs=30,
    initial_epoch=20
    # shuffle = False
)

Epoch 21/30
2194/2194 [==============================] - 55s 25ms/step - loss: 3.6762 - recall_at_10: 0.7259 - mrr_at_10: 0.7224 - ndcg_at_10: 0.7233 - map_at_10: 0.7224 - precision_at_10: 0.0726 - regularization_loss: 0.1679 - loss_batch: 3.6761 - val_loss: 3.6507 - val_recall_at_10: 0.6606 - val_mrr_at_10: 0.6419 - val_ndcg_at_10: 0.6467 - val_map_at_10: 0.6419 - val_precision_at_10: 0.0661 - val_regularization_loss: 0.1566 - val_loss_batch: 2.5545
Epoch 22/30
2194/2194 [==============================] - 58s 26ms/step - loss: 4.5342 - recall_at_10: 0.7213 - mrr_at_10: 0.6803 - ndcg_at_10: 0.6905 - map_at_10: 0.6803 - precision_at_10: 0.0721 - regularization_loss: 0.1464 - loss_batch: 4.5336 - val_loss: 3.9705 - val_recall_at_10: 0.0030 - val_mrr_at_10: 0.0030 - val_ndcg_at_10: 0.0030 - val_map_at_10: 0.0030 - val_precision_at_10: 3.0092e-04 - val_regularization_loss: 0.1468 - val_loss_batch: 2.5447
Epoch 23/30
2194/2194 [==============================] - 59s 27ms/step - loss: 4.1471 

In [17]:
model_v0.fit(
    train,
    validation_data=valid,
    schema=schema,
    batch_size=32,
    epochs=50,
    initial_epoch=30
    # shuffle = False
)

Epoch 31/50
2194/2194 [==============================] - 55s 25ms/step - loss: 7.2550 - recall_at_10: 0.6450 - mrr_at_10: 0.5660 - ndcg_at_10: 0.5853 - map_at_10: 0.5660 - precision_at_10: 0.0645 - regularization_loss: 0.1026 - loss_batch: 7.2530 - val_loss: 3.6137 - val_recall_at_10: 0.6239 - val_mrr_at_10: 0.6239 - val_ndcg_at_10: 0.6239 - val_map_at_10: 0.6239 - val_precision_at_10: 0.0624 - val_regularization_loss: 0.1196 - val_loss_batch: 2.5174
Epoch 32/50
2194/2194 [==============================] - 51s 23ms/step - loss: 9.6542 - recall_at_10: 0.6776 - mrr_at_10: 0.6344 - ndcg_at_10: 0.6450 - map_at_10: 0.6344 - precision_at_10: 0.0678 - regularization_loss: 0.1363 - loss_batch: 9.6508 - val_loss: 3.6436 - val_recall_at_10: 0.9446 - val_mrr_at_10: 0.9342 - val_ndcg_at_10: 0.9367 - val_map_at_10: 0.9342 - val_precision_at_10: 0.0945 - val_regularization_loss: 0.1495 - val_loss_batch: 2.5474
Epoch 33/50
2194/2194 [==============================] - 53s 24ms/step - loss: 4.8386 - re